<a href="https://colab.research.google.com/github/MuhammadShaheer-khan/Python-For-Finance-Stock-Analysis/blob/master/Amazon_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon short term stock price prediction Using FBprophet

In [31]:
# Import some important libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [32]:
# importing data of SP500

from google.colab import files
uploaded = files.upload()
import io
SP500_data = pd.read_csv(io.BytesIO(uploaded['all_stocks_5yr.csv']))

Saving all_stocks_5yr.csv to all_stocks_5yr (1).csv


In [33]:
# top 10 rows of data

SP500_data.head(10)

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
5,2013-02-15,13.93,14.61,13.93,14.50,15628000,AAL
6,2013-02-19,14.33,14.56,14.08,14.26,11354400,AAL
7,2013-02-20,14.17,14.26,13.15,13.33,14725200,AAL
8,2013-02-21,13.62,13.95,12.90,13.37,11922100,AAL
9,2013-02-22,13.57,13.60,13.21,13.57,6071400,AAL


In [34]:
# Information about data

SP500_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619040 entries, 0 to 619039
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    619040 non-null  object 
 1   open    619029 non-null  float64
 2   high    619032 non-null  float64
 3   low     619032 non-null  float64
 4   close   619040 non-null  float64
 5   volume  619040 non-null  int64  
 6   Name    619040 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 33.1+ MB


In [35]:
# sum all the null values

SP500_data.isnull().sum()

date       0
open      11
high       8
low        8
close      0
volume     0
Name       0
dtype: int64

In [36]:
# remove null data

SP500_data.dropna(inplace=True)

In [37]:
SP500_data.isnull().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
Name      0
dtype: int64

In [39]:
# setting date to year and month from date columns

SP500_data['year'] = SP500_data["date"].str.split("-", n = 2, expand = True)[0].astype(int)

SP500_data['month'] = SP500_data["date"].str.split("-", n = 2, expand = True)[1].astype(int)

In [40]:
SP500_data.head()

,date,open,high,low,close,volume,Name,year,month
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,2013,2
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,2013,2
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,2013,2
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,2013,2
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,2013,2


In [41]:
SP500_data["year"].unique()

array([2013, 2014, 2015, 2016, 2017, 2018])

In [42]:
# Extract AMZN data

amzn = SP500_data.loc[SP500_data['Name'] == 'AMZN']
amzn.head()

,date,open,high,low,close,volume,Name,year,month
46387,2013-02-08,261.40,265.25,260.555,261.95,3879078,AMZN,2013,2
46388,2013-02-11,263.20,263.25,256.600,257.21,3403403,AMZN,2013,2
46389,2013-02-12,259.19,260.16,257.000,258.70,2938660,AMZN,2013,2
46390,2013-02-13,261.53,269.96,260.300,269.47,5292996,AMZN,2013,2
46391,2013-02-14,267.37,270.65,265.400,269.24,3462780,AMZN,2013,2


In [43]:
# shape(Total rows and columns) of AMZN data

amzn.shape

(1259, 9)

In [46]:
# Extract data only year of 2k18

# yr_2018 = amzn.loc[amzn['year'] == '2014']
# yr_2018.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,date,open,high,low,close,volume,Name,year,month


In [48]:
amzn.tail()

,date,open,high,low,close,volume,Name,year,month
47641,2018-02-01,1445.00,1459.88,1385.14,1390.00,9113808,AMZN,2018,2
47642,2018-02-02,1477.39,1498.00,1414.00,1429.95,11125722,AMZN,2018,2
47643,2018-02-05,1402.62,1458.98,1320.72,1390.00,11494985,AMZN,2018,2
47644,2018-02-06,1361.46,1443.99,1351.79,1442.84,11066819,AMZN,2018,2
47645,2018-02-07,1449.00,1460.99,1415.15,1416.78,7162741,AMZN,2018,2


In [51]:
# display max row
pd.options.display.max_rows = 500000

# show data
amzn

,date,open,high,low,close,volume,Name,year,month
46387,2013-02-08,261.4000,265.2500,260.5550,261.950,3879078,AMZN,2013,2
46388,2013-02-11,263.2000,263.2500,256.6000,257.210,3403403,AMZN,2013,2
46389,2013-02-12,259.1900,260.1600,257.0000,258.700,2938660,AMZN,2013,2
46390,2013-02-13,261.5300,269.9600,260.3000,269.470,5292996,AMZN,2013,2
46391,2013-02-14,267.3700,270.6500,265.4000,269.240,3462780,AMZN,2013,2
46392,2013-02-15,267.6300,268.9200,263.1100,265.090,3979832,AMZN,2013,2
46393,2013-02-19,265.9100,270.1100,264.5000,269.750,2853752,AMZN,2013,2
46394,2013-02-20,270.2000,274.3000,266.3710,266.410,3528862,AMZN,2013,2
46395,2013-02-21,265.1200,269.4800,263.2500,265.940,3637396,AMZN,2013,2
46396,2013-02-22,266.6200,267.1100,261.6100,265.420,3123402,AMZN,2013,2


In [56]:
amzn_2k18 = 

,date,open,high,low,close,volume,Name,year,month
